In [1]:
from broker.execution_handler import OandaExecution
from strategies.donchian_breakout import DonchianBreakout
from broker.oanda_data_handler import OandaDataHandler
from portfolio import Portfolio
from settings import DOMAIN, ACCESS_TOKEN, OANDA_ACCOUNT_ID
from broker.account import OandaAccount
from live_trading import TradingEngine
from server.server import Server
import queue

In [2]:
heartbeat = 1
events = queue.Queue()
symbol_list = ['EUR_USD','AUD_USD']
risk = 0.02

account = OandaAccount(DOMAIN, ACCESS_TOKEN, OANDA_ACCOUNT_ID)

prices = OandaDataHandler(account,events,["EUR_USD"],'M1')
execution = OandaExecution(events_queue=events,account=account)

strategy_1 = DonchianBreakout(prices,events, entry_lookback=20, exit_lookback=20, atr_stop=3.,TP_atr=5.,name='DC_20x20')
strategy_2 = DonchianBreakout(prices,events, entry_lookback=50, exit_lookback=30, atr_stop=3.,TP_atr=5.,name='DC_50x30')
strategy_3 = DonchianBreakout(prices,events, entry_lookback=100, exit_lookback=50, atr_stop=3.,TP_atr=5.,name='DC_100x50')
strategy_4 = DonchianBreakout(prices,events, entry_lookback=200, exit_lookback=100, atr_stop=3.,TP_atr=5.,name='DC_200x100')
strategies = [strategy_1,strategy_2,strategy_3,strategy_4]

portfolio = Portfolio(events_queue=events, prices=prices,account=account, risk_per_trade = risk,strategies=strategies,open_trades=[])
server = Server(account,portfolio)

engine = TradingEngine(heartbeat,events,account,prices,execution,portfolio,strategies,server)

Strategy DC_20x20, lookback: 20x20, atr_stop 3.0
Strategy DC_50x30, lookback: 50x30, atr_stop 3.0
Strategy DC_100x50, lookback: 100x50, atr_stop 3.0
Strategy DC_200x100, lookback: 200x100, atr_stop 3.0
Socket created on port 5556


In [3]:
engine.start_live_trading()

merged with last candle
              datetime    open     high       low    close  volume complete
99 2016-07-19 15:30:00  1.1007  1.10108  1.100625  1.10093      37     True


In [4]:
for trade in portfolio.open_trades:
    print(trade.instrument,trade.pnl,trade.strategy,trade.units)

EUR_USD -0.53798 DC_20x20 727
EUR_USD -0.00781000000005 DC_20x20 781
EUR_USD -0.31008 DC_50x30 969
received event
Generating CloseEvent
               datetime     open     high      low    close  volume complete
129 2016-07-19 09:32:00  1.10696  1.10696  1.10684  1.10688       8     True


In [8]:
for strat in strategies:
    print(strat.invested,strat.identifier)

{'EUR_USD': 'OUT'} DC_20x20
{'EUR_USD': 'LONG'} DC_50x30
{'EUR_USD': 'OUT'} DC_100x50
{'EUR_USD': 'OUT'} DC_200x100
               datetime    open     high      low    close  volume complete
130 2016-07-19 09:33:00  1.1069  1.10692  1.10682  1.10683      11     True
received event
Data: Received MARKET_ORDER_CREATE Event,not yet supported
received event
Data: Received MARKET_ORDER_CREATE Event,not yet supported
{'instrument': 'EUR_USD', 'tradeReduced': {}, 'price': 1.10673, 'tradesClosed': [{'units': 781, 'id': 10386258070, 'side': 'buy'}], 'time': '2016-07-19T09:34:21.000000Z', 'tradeOpened': {'units': 219, 'side': 'sell', 'takeProfit': 0, 'trailingStop': 0, 'stopLoss': 1.1073, 'id': 10386271165}}
